In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
from sklearn.metrics import classification_report,confusion_matrix

# Path to image folders, 
data_path = fr'../Datasets/full/Training'

def remove_white_background(pixels):
    newPixels = []
    for pixel in pixels:
        pixel = list(pixel)
        if ((256 > pixel[0] > 200) and (256 > pixel[1] > 200) and (256 > pixel[2] > 200)):
            pixel[0] = 0
            pixel[1] = 0
            pixel[2] = 0
        newPixels.append(pixel)
    
    return newPixels


def redify(pixels):
    return [r for r, g, b in pixels]

                
def greenify(pixels):
    return [g for r, g, b in pixels]


def blueify(pixels):
    return [b for r, g, b in pixels]


def get_rgb_pixels_onehot_labels(src):
    print("Starting...")
    newPixels = []
    y = np.empty(shape=[0, 1])

    for subdir in os.listdir(src):
        current_path = os.path.join(src, subdir)
        for file in os.listdir(current_path):
            img = Image.open(os.path.join(current_path, file))
            imgResize = img.resize((24,24))
            pixels = list(imgResize.getdata())
            pixels = remove_white_background(pixels)
            newPixels.append(pixels)
            y = np.append(y, subdir)
    return newPixels, LabelBinarizer().fit_transform(y) # OneHot encode y


def process_files(src):
    X_red_train = []
    X_red_validation = []
    X_red_test = []
    X_green_train = []
    X_green_validation = []
    X_green_test = []
    X_blue_train = []
    X_blue_validation = []
    X_blue_test = []
    all_pixels, y = get_rgb_pixels_onehot_labels(src)

    X_train, X_validation, y_train, y_validation = train_test_split(all_pixels, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)
    
    for pixels in X_train:       
        X_red_train.append(redify(pixels.copy()))
        X_green_train.append(greenify(pixels.copy()))
        X_blue_train.append(blueify(pixels.copy()))
        
    for pixels in X_validation:       
        X_red_validation.append(redify(pixels.copy()))
        X_green_validation.append(greenify(pixels.copy()))
        X_blue_validation.append(blueify(pixels.copy()))
        
    for pixels in X_test:       
        X_red_test.append(redify(pixels.copy()))
        X_green_test.append(greenify(pixels.copy()))
        X_blue_test.append(blueify(pixels.copy()))
    
    
    print("Finished \n")
    return np.asarray(X_red_train), np.asarray(X_red_validation), np.asarray(X_red_test), np.asarray(X_green_train), np.asarray(X_green_validation), np.asarray(X_green_test), np.asarray(X_blue_train), np.asarray(X_blue_validation), np.asarray(X_blue_test), y_train, y_validation, y_test


def get_youdens_index(predictions, Y):
    # Calculate true positive/negative and false positive/negative
    tp = sum((Y == predictions) * (Y == 1) * 1)
    tn = sum((Y == predictions) * (Y == 0) * 1)
    fp = sum((Y != predictions) * (Y == 0) * 1)
    fn = sum((Y != predictions) * (Y == 1) * 1)
    
    # Calculate sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (fp + tn)
    
    result = sensitivity - (1 - specificity)
    # Put it in a dateframe for nicer visuals
    df = pd.DataFrame({'Youdens Index': result})
    pd.set_option('display.max_rows', 200)
    
    return df

In [2]:
import time

# Call process_files and assign variables
X_red_train, X_red_validation, X_red_test, X_green_train, X_green_validation, X_green_test, X_blue_train, X_blue_validation, X_blue_test, Y_train, Y_validation, Y_test = process_files(data_path)
# Fit/train train-datasets and store prediction vectors in variables
from sklearn import svm
from sklearn.neural_network import MLPClassifier

# Red Config HL Test
print("Red Config HL Tests")
for hl in [(432), (216), (216, 108), (108), (108, 54)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl)
    t0 = time.time()
    mlp.fit(X_red_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_red_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_red_validation), Y_validation), "\n")

# Green Config HL Test
print("Green Config HL Tests")
for hl in [(432), (216), (216, 108), (108), (108, 54)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl)
    t0 = time.time()
    mlp.fit(X_green_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_green_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_green_validation), Y_validation), "\n")
    
# Blue Config HL Test
print("Blue Config HL Tests")
for hl in [(432), (216), (216, 108), (108), (108, 54)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl)
    t0 = time.time()
    mlp.fit(X_blue_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_blue_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_blue_validation), Y_validation), "\n")

Starting...
Finished 

Red Config HL Tests
Hidden layers:  432
Score:  0.9238694586767654 Time:  217.7574245929718
     Youdens Index
0         0.908397
1         0.991485
2         0.921875
3         0.900707
4         0.999215
5         0.984621
6         0.991691
7         0.976538
8         0.892737
9         0.965217
10        0.954030
11        0.740458
12        0.955056
13        0.679277
14        0.973684
15        0.954142
16        0.961776
17        0.950357
18        0.938931
19        0.775000
20        1.000000
21        0.969241
22        0.999776
23        0.991469
24        0.999944
25        0.962511
26        1.000000
27        0.958659
28        0.974113
29        1.000000
30        1.000000
31        0.923608
32        0.983415
33        0.992366
34        0.922318
35        0.957661
36        0.934903
37        0.999832
38        0.880000
39        0.923608
40        0.904000
41        0.914439
42        0.999608
43        0.954198
44        0.992395
45        0

Score:  0.8580975718422811 Time:  115.58892178535461
     Youdens Index
0         0.961439
1         0.991485
2         0.866514
3         0.807758
4         0.620043
5         0.961439
6         0.983438
7         0.625449
8         0.816289
9         0.912427
10        0.846936
11        0.967839
12        0.970898
13        0.404019
14        0.929712
15        0.900651
16        0.969241
17        0.933604
18        0.968961
19        0.891050
20        0.983740
21        0.770319
22        0.976763
23        0.945948
24        0.930120
25        0.892429
26        0.992142
27        0.895943
28        0.948619
29        1.000000
30        0.999776
31        0.953806
32        0.850735
33        0.885047
34        0.869387
35        0.940041
36        0.902327
37        0.942801
38        0.647832
39        0.930681
40        0.944000
41        0.967633
42        0.835163
43        0.953974
44        0.984734
45        0.815784
46        0.946340
47        0.915918
48        0.9847

Score:  0.7617509467587436 Time:  107.81178998947144
     Youdens Index
0         0.984508
1         0.991092
2         0.194527
3         0.541087
4         0.774633
5         0.922262
6         0.982149
7         0.960430
8         0.884486
9         0.911418
10        0.960149
11        0.915357
12        0.926741
13        0.617816
14        0.893616
15        0.846879
16        0.930681
17        0.917187
18        0.953637
19        0.591330
20        0.951051
21        0.747362
22        0.960037
23        0.853223
24        0.838965
25        0.923277
26        0.999944
27        0.992840
28        0.849526
29        0.999888
30        1.000000
31        0.992142
32        0.857373
33        0.838348
34        0.417659
35        0.856820
36        0.796524
37        0.827619
38        0.927103
39        0.908173
40        0.950205
41        0.994033
42        0.670495
43        0.785586
44        0.972988
45        0.824203
46        0.975641
47        0.989898
48        0.9686

Score:  0.9467030519046559 Time:  97.59988474845886
     Youdens Index
0         0.992366
1         0.999495
2         0.991683
3         0.862595
4         0.999607
5         0.999944
6         0.999776
7         0.923440
8         0.984677
9         0.991304
10        0.915974
11        0.992310
12        0.966292
13        1.000000
14        0.964856
15        0.908285
16        0.984565
17        0.917355
18        0.977099
19        0.791555
20        0.999888
21        0.938931
22        0.870229
23        1.000000
24        0.931242
25        0.935829
26        1.000000
27        0.999944
28        0.888043
29        0.992310
30        0.999776
31        1.000000
32        0.801653
33        0.992366
34        0.877358
35        0.941274
36        0.959237
37        0.923810
38        1.000000
39        0.816569
40        0.959607
41        0.967577
42        0.784810
43        0.961832
44        0.996141
45        0.984452
46        0.976763
47        0.999944
48        0.97704

In [3]:
# default alpha=0.0001

print("Red Config Alpha Tests")
# Red Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=108, alpha=a, max_iter=300)
    t0 = time.time()
    mlp.fit(X_red_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_red_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_red_validation), Y_validation), "\n")
    
print("Green Config Alpha Tests")
# Green Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=108, alpha=a, max_iter=300)
    t0 = time.time()
    mlp.fit(X_green_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_green_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_green_validation), Y_validation), "\n")
    
print("Blue Config Alpha Tests")
# Blue Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=108, alpha=a, max_iter=300)
    t0 = time.time()
    mlp.fit(X_blue_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_blue_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_blue_validation), Y_validation), "\n")    

Red Config Alpha Tests
Alpha:  0.0001
Score:  0.5944531075963466 Time:  76.85132575035095
     Youdens Index
0         0.152503
1         0.829242
2         0.140232
3         0.349630
4         0.054207
5         0.852998
6         0.875423
7         0.175348
8         0.455771
9         0.582272
10        0.722891
11        0.632746
12        0.578202
13        0.053379
14        0.938372
15        0.709250
16        0.922879
17        0.874856
18        0.854457
19        0.740041
20        0.967311
21        0.755052
22        0.763134
23        0.417435
24        0.731366
25        0.860456
26        0.999888
27        0.054599
28        0.591994
29        1.000000
30        0.953862
31        0.861978
32        0.759209
33        0.602661
34        0.563820
35        0.716106
36        0.788450
37        0.761120
38        0.543776
39        0.892625
40        0.927888
41        0.956206
42        0.442311
43        0.844748
44        0.946768
45        0.297542
46        0.82308

Score:  0.8010692804633549 Time:  66.69290280342102
     Youdens Index
0         0.960317
1         0.981792
2         0.874383
3         0.511450
4         0.925633
5         0.960317
6         0.950427
7         0.717108
8         0.496127
9         0.955961
10        0.778458
11        0.937248
12        0.752809
13        0.737204
14        0.999664
15        0.923664
16        0.922598
17        0.834543
18        0.900427
19        0.791386
20        0.861789
21        0.786260
22        0.981872
23        0.854233
24        0.976819
25        0.716578
26        0.999944
27        0.654822
28        0.629104
29        0.999888
30        0.999888
31        0.800966
32        0.801541
33        0.953974
34        0.762349
35        0.799551
36        0.837230
37        0.933165
38        0.862318
39        0.907499
40        0.911327
41        0.961216
42        0.734121
43        0.908060
44        0.992282
45        0.929839
46        0.625786
47        0.991188
48        0.99219

Score:  0.8585987970594787 Time:  81.36665534973145
     Youdens Index
0         0.992142
1         0.991316
2         0.795473
3         0.867592
4         0.960118
5         0.998878
6         0.991299
7         0.961552
8         0.961495
9         0.998655
10        0.801527
11        0.900763
12        0.971348
13        0.869724
14        0.973404
15        0.900483
16        0.938482
17        0.851240
18        0.885440
19        0.741218
20        0.951220
21        0.899866
22        0.946228
23        0.938763
24        0.967895
25        0.998593
26        0.984677
27        0.989566
28        0.968023
29        0.999944
30        1.000000
31        0.999551
32        0.826166
33        0.999719
34        0.740290
35        0.916555
36        0.909504
37        0.904594
38        0.863720
39        0.869612
40        0.887832
41        0.978441
42        0.898622
43        0.900651
44        0.992056
45        0.847160
46        0.907724
47        0.954086
48        0.99186

Score:  0.8751392292269994 Time:  88.96292185783386
     Youdens Index
0         0.931242
1         0.999664
2         0.827788
3         0.900539
4         0.937760
5         0.992198
6         0.909836
7         0.923496
8         0.976875
9         0.904348
10        0.884374
11        0.923328
12        0.977303
13        0.892961
14        0.903509
15        0.854401
16        0.961608
17        0.809917
18        0.900427
19        0.699383
20        0.999439
21        0.892681
22        0.905816
23        0.823979
24        0.953862
25        0.962398
26        0.999888
27        0.958828
28        0.715342
29        1.000000
30        1.000000
31        0.992366
32        0.743577
33        0.816794
34        0.640773
35        0.907997
36        0.869638
37        0.951877
38        1.000000
39        0.808768
40        0.863832
41        0.939882
42        0.859529
43        0.816682
44        0.931559
45        0.847272
46        0.929390
47        0.999719
48        0.95386

In [4]:
# Red
mlpc_red =  MLPClassifier(hidden_layer_sizes=(108), activation='relu', solver='adam', random_state=1)
mlpc_red.fit(X_red_train, Y_train)
vector_red = mlpc_red.predict(X_red_train)

# Green
mlpc_green = MLPClassifier(hidden_layer_sizes=(108), activation='relu', solver='adam', random_state=1)
mlpc_green.fit(X_green_train, Y_train)
vector_green = mlpc_green.predict(X_green_train)

# Blue
mlpc_blue = MLPClassifier(hidden_layer_sizes=(108), activation='relu', solver='adam', random_state=1)
mlpc_blue.fit(X_blue_train, Y_train)
vector_blue = mlpc_blue.predict(X_blue_train)

# Validation
vector_red_val = mlpc_red.predict(X_red_validation)
vector_green_val = mlpc_green.predict(X_green_validation)
vector_blue_val = mlpc_blue.predict(X_blue_validation)


print("Red Model Validation")
red_val_predict = mlpc_red.predict(X_red_validation)
print(classification_report(Y_validation, red_val_predict, zero_division=0))
print(get_youdens_index(red_val_predict, Y_validation), "\n")

print("Green Model Validation")
green_val_predict = mlpc_green.predict(X_green_validation)
print(classification_report(Y_validation, green_val_predict, zero_division=0))
print(get_youdens_index(green_val_predict, Y_validation), "\n")

print("Blue Model Validation")
blue_val_predict = mlpc_blue.predict(X_blue_validation)
print(classification_report(Y_validation, blue_val_predict, zero_division=0))
print(get_youdens_index(blue_val_predict, Y_validation), "\n")

c:\users\olive\documents\6. semester\applied artificial intelligence\ai_exam\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\users\olive\documents\6. semester\applied artificial intelligence\ai_exam\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Red Model Validation
              precision    recall  f1-score   support

           0       0.76      0.28      0.41       131
           1       0.84      0.87      0.85       119
           2       0.00      0.00      0.00       128
           3       0.62      0.37      0.47       131
           4       0.00      0.00      0.00       129
           5       0.94      0.38      0.54       131
           6       0.79      0.91      0.84       122
           7       0.67      0.14      0.23       131
           8       0.00      0.00      0.00       131
           9       0.63      0.50      0.55       115
          10       0.71      0.37      0.49       131
          11       0.58      0.39      0.47       131
          12       0.82      0.56      0.66       178
          13       0.00      0.00      0.00       131
          14       0.92      0.89      0.90       114
          15       0.78      0.82      0.80       131
          16       0.87      0.73      0.80       131
      

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       131
           1       0.98      0.76      0.85       119
           2       0.48      0.16      0.24       128
           3       0.90      0.66      0.76       131
           4       0.81      0.88      0.84       129
           5       0.96      0.67      0.79       131
           6       0.84      0.96      0.89       122
           7       0.84      0.77      0.80       131
           8       0.88      0.44      0.59       131
           9       0.87      0.82      0.84       115
          10       0.83      0.95      0.88       131
          11       0.94      0.94      0.94       131
          12       0.92      0.96      0.94       178
          13       0.84      0.79      0.81       131
          14       0.94      0.75      0.83       114
          15       0.93      0.53      0.68       131
          16       0.87      0.88      0.87       131
          17       0.86    

     Youdens Index
0         0.984116
1         0.983137
2         0.873934
3         0.831724
4         0.999046
5         0.755725
6         0.966989
7         0.930176
8         0.968961
9         0.956241
10        0.845701
11        0.953694
12        0.926798
13        0.969185
14        0.999888
15        0.770487
16        0.900483
17        0.917131
18        0.914965
19        0.740265
20        0.983459
21        0.868714
22        0.914179
23        0.663617
24        0.922823
25        0.967408
26        0.977099
27        0.963679
28        0.658998
29        0.961832
30        1.000000
31        0.976931
32        0.883737
33        0.969017
34        0.640829
35        0.833277
36        0.933950
37        0.865770
38        0.991944
39        0.755613
40        0.935664
41        0.891585
42        0.746108
43        0.816233
44        0.934909
45        0.861922
46        0.823698
47        0.998990
48        0.991020
49        0.928571
50        0.999215
51        0.

In [5]:
# Stack the rgb predictions to get combi model values 
X_combined_train = np.column_stack((vector_red, vector_green, vector_blue)) # shape (1745, 18) - 18 features
X_combined_val = np.column_stack((vector_red_val, vector_green_val, vector_blue_val)) # shape (582, 18) - 18 features

# Combi Config HL Test
print("Combi Config HL Tests")
for hl in [(18), (14), (14, 7)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl, max_iter=1000)
    t0 = time.time()
    mlp.fit(X_combined_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_combined_val, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_combined_val), Y_validation), "\n")


print("Combi Config Alpha Tests")
# Combi Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=108, alpha=a, max_iter=300)
    t0 = time.time()
    mlp.fit(X_combined_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_combined_val, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_combined_val), Y_validation), "\n")

Combi Config HL Tests
Hidden layers:  18
Score:  0.9228670082423702 Time:  84.04330205917358
     Youdens Index
0         0.977099
1         0.991597
2         0.835601
3         0.877470
4         0.976632
5         0.908285
6         0.975354
7         0.931073
8         0.969353
9         0.947602
10        0.923271
11        0.976987
12        0.938202
13        0.946565
14        0.991116
15        0.892905
16        0.900595
17        0.925620
18        0.877863
19        0.724664
20        1.000000
21        0.877358
22        0.992198
23        0.946228
24        0.900371
25        0.935660
26        1.000000
27        0.979470
28        0.913311
29        0.992366
30        1.000000
31        0.984733
32        0.793388
33        0.977099
34        0.724518
35        0.841611
36        0.926437
37        0.780896
38        0.984000
39        0.892737
40        0.944000
41        0.930481
42        0.784363
43        0.877694
44        0.988593
45        0.900539
46        0.93

Score:  0.9314992203163288 Time:  61.58111047744751
     Youdens Index
0         0.984733
1         0.991541
2         0.843301
3         0.877526
4         0.984384
5         0.908341
6         0.991803
7         0.946284
8         0.954030
9         0.965049
10        0.938595
11        0.984621
12        0.966292
13        0.969410
14        0.973572
15        0.907724
16        0.946397
17        0.925620
18        0.885440
19        0.732941
20        1.000000
21        0.877358
22        0.992254
23        0.969073
24        0.915862
25        0.978272
26        1.000000
27        0.989341
28        0.928371
29        0.992366
30        1.000000
31        1.000000
32        0.859392
33        0.984677
34        0.747362
35        0.841611
36        0.918138
37        0.828571
38        0.991944
39        0.892737
40        0.944000
41        0.935829
42        0.746388
43        0.900651
44        0.996198
45        0.893018
46        0.953862
47        0.999944
48        0.98473

In [6]:
# Combi Train
mlpc_combi =  MLPClassifier(hidden_layer_sizes=(18), activation='relu', solver='adam', random_state=1, max_iter=300)
mlpc_combi.fit(X_combined_train, Y_train)

# Validation
print("Combined Model Validation")
combi_val = mlpc_combi.predict(X_combined_val)
print(classification_report(Y_validation, combi_val, zero_division=0))
print(get_youdens_index(combi_val, Y_validation), "\n")

Combined Model Validation
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       131
           1       0.99      0.99      0.99       119
           2       0.95      0.84      0.89       128
           3       0.96      0.87      0.91       131
           4       1.00      0.97      0.98       129
           5       0.98      0.92      0.95       131
           6       0.98      0.98      0.98       122
           7       0.98      0.95      0.96       131
           8       1.00      0.93      0.96       131
           9       0.96      0.93      0.95       115
          10       0.94      0.92      0.93       131
          11       0.98      0.98      0.98       131
          12       1.00      0.96      0.98       178
          13       0.99      0.96      0.98       131
          14       0.99      0.96      0.97       114
          15       0.90      0.89      0.89       131
          16       0.98      0.91      0.94       131
 

In [7]:
full_data_path = fr'../Datasets/full/Training'
X_red_train_final, X_red_validation_final, X_red_test_final, X_green_train_final, X_green_validation_final, X_green_test_final, X_blue_train_final, X_blue_validation_final, X_blue_test_final, Y_train_final, Y_validation_final, Y_test_final = process_files(full_data_path)

Starting...
Finished 



In [8]:
# Red Train Final
mlpc_red_final =  MLPClassifier(hidden_layer_sizes=(108), activation='relu', solver='adam', random_state=1, max_iter=300)
mlpc_red_final.fit(X_red_train_final, Y_train_final)
vector_red_final = mlpc_red_final.predict(X_red_train_final)

# Green Final
mlpc_green_final = MLPClassifier(hidden_layer_sizes=(108), activation='relu', solver='adam', random_state=1, max_iter=300)
mlpc_green_final.fit(X_green_train_final, Y_train_final)
vector_green_final = mlpc_green_final.predict(X_green_train_final)

# Blue Final
mlpc_blue_final = MLPClassifier(hidden_layer_sizes=(108), activation='relu', solver='adam', random_state=1, max_iter=300)
mlpc_blue_final.fit(X_blue_train_final, Y_train_final)
vector_blue_final = mlpc_blue_final.predict(X_blue_train_final)

# Validation Final
vector_red_val_final = mlpc_red_final.predict(X_red_validation_final)
vector_green_val_final = mlpc_green_final.predict(X_green_validation_final)
vector_blue_val_final = mlpc_blue_final.predict(X_blue_validation_final)


print("Red Model Validation Final")
red_val_predict_final = mlpc_red_final.predict(X_red_validation_final)
print(classification_report(Y_validation_final, red_val_predict_final, zero_division=0))
print(get_youdens_index(red_val_predict_final, Y_validation_final), "\n")

print("Green Model Validation Final")
green_val_predict_final = mlpc_green_final.predict(X_green_validation_final)
print(classification_report(Y_validation_final, green_val_predict_final, zero_division=0))
print(get_youdens_index(green_val_predict_final, Y_validation_final), "\n")

print("Blue Model Validation Final")
blue_val_predict_final = mlpc_blue_final.predict(X_blue_validation_final)
print(classification_report(Y_validation_final, blue_val_predict_final, zero_division=0))
print(get_youdens_index(blue_val_predict_final, Y_validation_final), "\n")

Red Model Validation Final
              precision    recall  f1-score   support

           0       0.76      0.28      0.41       131
           1       0.84      0.87      0.85       119
           2       0.00      0.00      0.00       128
           3       0.62      0.37      0.47       131
           4       0.00      0.00      0.00       129
           5       0.94      0.38      0.54       131
           6       0.79      0.91      0.84       122
           7       0.67      0.14      0.23       131
           8       0.00      0.00      0.00       131
           9       0.63      0.50      0.55       115
          10       0.71      0.37      0.49       131
          11       0.58      0.39      0.47       131
          12       0.82      0.56      0.66       178
          13       0.00      0.00      0.00       131
          14       0.92      0.89      0.90       114
          15       0.78      0.82      0.80       131
          16       0.87      0.73      0.80       131


              precision    recall  f1-score   support

           0       0.95      0.99      0.97       131
           1       0.97      0.97      0.97       119
           2       0.87      0.10      0.18       128
           3       0.88      0.69      0.77       131
           4       0.82      0.88      0.85       129
           5       0.78      1.00      0.88       131
           6       0.91      0.84      0.88       122
           7       0.93      0.76      0.83       131
           8       0.75      0.91      0.82       131
           9       0.96      0.69      0.80       115
          10       0.96      0.90      0.93       131
          11       0.92      0.91      0.92       131
          12       0.93      0.90      0.91       178
          13       0.82      0.81      0.82       131
          14       0.92      0.93      0.93       114
          15       0.93      0.68      0.78       131
          16       0.93      0.81      0.87       131
          17       0.88    

     Youdens Index
0         0.976875
1         0.999944
2         0.881747
3         0.846711
4         0.999215
5         0.976987
6         0.934034
7         0.953076
8         0.969073
9         0.886844
10        0.694152
11        0.892513
12        0.943820
13        0.999832
14        0.999888
15        0.785699
16        0.915750
17        0.917131
18        0.885216
19        0.805025
20        0.999383
21        0.808992
22        0.979908
23        0.967951
24        0.930793
25        0.962060
26        0.977099
27        0.968642
28        0.809761
29        0.984733
30        1.000000
31        0.992198
32        0.851128
33        0.984340
34        0.534071
35        0.808277
36        0.917746
37        0.818655
38        0.992000
39        0.816626
40        0.935888
41        0.906333
42        0.531310
43        0.890998
44        0.934966
45        0.885103
46        0.957568
47        0.997980
48        0.953862
49        0.967917
50        0.992310
51        0.

In [9]:
# Stack the final rgb predictions to get final combi model values 
X_combined_train_final = np.column_stack((vector_red_final, vector_green_final, vector_blue_final)) # shape (1745, 18) - 18 features
X_combined_val_final = np.column_stack((vector_red_val_final, vector_green_val_final, vector_blue_val_final)) # shape (582, 18) - 18 features

# Combi Train Final
mlpc_combi_final =  MLPClassifier(hidden_layer_sizes=(18), activation='relu', solver='adam', random_state=1, max_iter=300)
mlpc_combi_final.fit(X_combined_train_final, Y_train_final)

# Validation Final
print("Combined Model Validation")
combi_val_final = mlpc_combi_final.predict(X_combined_val_final)
print(classification_report(Y_validation_final, combi_val_final, zero_division=0))
print(get_youdens_index(combi_val_final, Y_validation_final), "\n")

Combined Model Validation
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       131
           1       1.00      0.99      1.00       119
           2       0.93      0.88      0.91       128
           3       0.97      0.89      0.93       131
           4       0.98      0.97      0.98       129
           5       0.98      0.96      0.97       131
           6       1.00      0.97      0.98       122
           7       0.97      0.96      0.97       131
           8       0.99      0.97      0.98       131
           9       1.00      0.92      0.96       115
          10       1.00      0.93      0.96       131
          11       0.98      0.97      0.98       131
          12       0.99      0.97      0.98       178
          13       1.00      1.00      1.00       131
          14       0.99      0.97      0.98       114
          15       0.98      0.91      0.94       131
          16       0.99      0.90      0.94       131
 

In [10]:
vector_red_test_final = mlpc_red_final.predict(X_red_test_final)
vector_green_test_final = mlpc_green_final.predict(X_green_test_final)
vector_blue_test_final = mlpc_blue_final.predict(X_blue_test_final)

print("Red Model Test Final")
print(classification_report(Y_test_final, vector_red_test_final, zero_division=0))
print(get_youdens_index(vector_red_test_final, Y_test_final), "\n")

print("Green Model Test Final")
print(classification_report(Y_test_final, vector_green_test_final, zero_division=0))
print(get_youdens_index(vector_green_test_final, Y_test_final), "\n")

print("Blue Model Test Final")
print(classification_report(Y_test_final, vector_blue_test_final, zero_division=0))
print(get_youdens_index(vector_blue_test_final, Y_test_final), "\n")

Red Model Test Final
              precision    recall  f1-score   support

           0       0.64      0.27      0.39       131
           1       0.83      0.90      0.86       118
           2       0.00      0.00      0.00       128
           3       0.65      0.47      0.55       131
           4       0.00      0.00      0.00       128
           5       0.92      0.35      0.51       132
           6       0.79      0.91      0.85       122
           7       0.66      0.15      0.24       131
           8       0.00      0.00      0.00       131
           9       0.54      0.50      0.52       115
          10       0.73      0.40      0.51       131
          11       0.55      0.40      0.46       131
          12       0.79      0.60      0.69       179
          13       0.00      0.00      0.00       131
          14       0.89      0.78      0.83       114
          15       0.79      0.79      0.79       132
          16       0.80      0.66      0.72       131
      

     Youdens Index
0         0.999495
1         0.974576
2         0.077957
3         0.830883
4         0.857748
5         0.982773
6         0.933978
7         0.778289
8         0.958859
9         0.695428
10        0.908173
11        0.900034
12        0.915526
13        0.776887
14        0.859033
15        0.628563
16        0.869556
17        0.842639
18        0.885216
19        0.540489
20        0.959069
21        0.716940
22        0.427425
23        0.913562
24        0.792435
25        0.694793
26        0.992310
27        0.943768
28        0.744053
29        0.992310
30        0.992366
31        0.999776
32        0.759602
33        0.778345
34        0.616862
35        0.782661
36        0.699187
37        0.874776
38        0.895551
39        0.854289
40        0.919944
41        0.850099
42        0.695084
43        0.763022
44        0.980577
45        0.770768
46        0.823866
47        0.663842
48        0.938370
49        0.817180
50        0.434722
51        0.

In [11]:
X_combined_test_final = np.column_stack((vector_red_test_final, vector_green_test_final, vector_blue_test_final)) # shape (582, 18) - 18 features

# Test Final
print("Combined Model Test")
combi_test_final = mlpc_combi_final.predict(X_combined_test_final)
print(classification_report(Y_test_final, combi_test_final, zero_division=0))
print(get_youdens_index(combi_test_final, Y_test_final), "\n")

Combined Model Test
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       131
           1       0.99      0.99      0.99       118
           2       0.93      0.88      0.90       128
           3       0.97      0.96      0.97       131
           4       0.98      0.95      0.97       128
           5       0.96      0.99      0.98       132
           6       1.00      0.98      0.99       122
           7       0.99      0.95      0.97       131
           8       0.98      0.95      0.97       131
           9       0.99      0.91      0.95       115
          10       1.00      0.94      0.97       131
          11       0.99      0.97      0.98       131
          12       0.99      0.96      0.97       179
          13       1.00      0.98      0.99       131
          14       0.99      0.94      0.96       114
          15       0.98      0.90      0.94       132
          16       0.98      0.97      0.98       131
       